In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


__DATA EXPLORATION__

In [3]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df.shape

(891, 12)

__checking missing values__

In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

__Handling missing values__

In [6]:
mode = df.mode()
mode.iloc[0]

PassengerId                      1
Survived                       0.0
Pclass                         3.0
Name           Abbing, Mr. Anthony
Sex                           male
Age                           24.0
SibSp                          0.0
Parch                          0.0
Ticket                        1601
Fare                          8.05
Cabin                      B96 B98
Embarked                         S
Name: 0, dtype: object

In [7]:
df.fillna(mode.iloc[0], inplace = True)

In [8]:
df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

__checking if i have categorical variables__

In [9]:
df.describe(include=object)

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,891,891
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,691,646


In [10]:
#since machine learning models cannot understand numbers , so we use one hot encoding
#based on the dataset too  it seems that Pclass needs one hot encoding too
#lest find out

df['Pclass'].unique()

array([3, 1, 2])

In [11]:
df = pd.get_dummies(df, columns = ['Sex', 'Embarked','Pclass'], dtype = int)
df.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,B96 B98,0,1,0,0,1,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1,0,1,0,0,1,0,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,1,0,0,0,1,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,0,0,0,1,1,0,0
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,B96 B98,0,1,0,0,1,0,0,1


__check the non-categorical part and see if there is anything you need to do__

In [12]:
df.describe(include = np.number)

,PassengerId,Survived,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,28.566970,0.523008,0.381594,32.204208,0.352413,0.647587,0.188552,0.086420,0.725028,0.242424,0.206510,0.551066
std,257.353842,0.486592,13.199572,1.102743,0.806057,49.693429,0.477990,0.477990,0.391372,0.281141,0.446751,0.428790,0.405028,0.497665
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,24.000000,0.000000,0.000000,14.454200,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,668.500000,1.000000,35.000000,1.000000,0.000000,31.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,891.000000,1.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
#the Fare dataset looks weird is long-tailed.
#ML models do not do well with long-tailed distribution, lets retool it.
#generally , with ML models ,we log things that salary, population that has tendencies for exponential growth. 
#just check and the distribution and you can decide or not 
#we addingn plus 1 to the log because the log of zero is infinity and we dont want to have issues with that.

df['LogFare'] = np.log(df['Fare'] + 1) 

In [14]:
df.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,LogFare
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,B96 B98,0,1,0,0,1,0,0,1,2.110213
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1,0,1,0,0,1,0,0,4.280593
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,1,0,0,0,1,0,0,1,2.188856
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,0,0,0,1,1,0,0,3.990834
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,B96 B98,0,1,0,0,1,0,0,1,2.202765


__I love using Pytorch a lot__

In [15]:
from torch import tensor

In [16]:
t_dep = tensor(df['Survived'])

In [17]:
t_dep.shape

torch.Size([891])

In [18]:
df.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q',
       'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'LogFare'],
      dtype='object')

In [19]:
added_cols = ['Sex_female','Sex_male', 'Embarked_C', 'Embarked_Q',
       'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3']

In [20]:
indep = ['Age', 'SibSp', 'Parch', 'LogFare'] + added_cols
indep

['Age',
 'SibSp',
 'Parch',
 'LogFare',
 'Sex_female',
 'Sex_male',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3']

In [21]:
len(indep)

12

In [22]:
import torch

In [23]:
t_indeps = tensor(df[indep].values, dtype=torch.float)
t_indeps.shape

torch.Size([891, 12])

__LINEAR REGRESION MODEL__

__#1. CALCULATE PREDICTIONS__

In [25]:
#but to do that we need the coefficients or weight or params  , whatever you call it. 
#since every feature must have a weight 
# i subtracted the (-0.5) so can shift the values generated from the normal distribution a bit to the left, we have a 

n_coef = t_indeps.shape[1]
coef = torch.randn(n_coef)-0.5
coef



tensor([-0.2148, -1.7572, -0.8034, -0.7240,  0.4958,  0.0252, -1.6377, -0.6625,
        -1.1374,  0.0711,  1.8761,  0.6024])

In [26]:
#now I can calculate prediction
#whats happening here is broadcasting , it is part if what we enjoy in pytorch

t_indeps * coef

tensor([[-4.7252, -1.7572, -0.0000,  ...,  0.0000,  0.0000,  0.6024],
        [-8.1617, -1.7572, -0.0000,  ...,  0.0711,  0.0000,  0.0000],
        [-5.5843, -0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.6024],
        ...,
        [-5.1548, -1.7572, -1.6068,  ...,  0.0000,  0.0000,  0.6024],
        [-5.5843, -0.0000, -0.0000,  ...,  0.0711,  0.0000,  0.0000],
        [-6.8730, -0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.6024]])

In [27]:
df[indep].head()

,Age,SibSp,Parch,LogFare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,22.0,1,0,2.110213,0,1,0,0,1,0,0,1
1,38.0,1,0,4.280593,1,0,1,0,0,1,0,0
2,26.0,0,0,2.188856,1,0,0,0,1,0,0,1
3,35.0,1,0,3.990834,1,0,0,0,1,1,0,0
4,35.0,0,0,2.202765,0,1,0,0,1,0,0,1


In [28]:
#checking out things 
values , indices = (t_indeps).max(dim=0)
values

tensor([80.0000,  8.0000,  6.0000,  6.2409,  1.0000,  1.0000,  1.0000,  1.0000,
         1.0000,  1.0000,  1.0000,  1.0000])

In [29]:
#notice that age is slightly higher...we can normalize because we dont our ML model to rate age more than other features
#across all  features find the highest value 

values , indices = (t_indeps).max(dim=0)
t_indeps = t_indeps/values
t_indeps

tensor([[0.2750, 0.1250, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.4750, 0.1250, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.3250, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        ...,
        [0.3000, 0.1250, 0.3333,  ..., 0.0000, 0.0000, 1.0000],
        [0.3250, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.4000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000]])

In [30]:
t_indeps * coef

tensor([[-0.0591, -0.2196, -0.0000,  ...,  0.0000,  0.0000,  0.6024],
        [-0.1020, -0.2196, -0.0000,  ...,  0.0711,  0.0000,  0.0000],
        [-0.0698, -0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.6024],
        ...,
        [-0.0644, -0.2196, -0.2678,  ...,  0.0000,  0.0000,  0.6024],
        [-0.0698, -0.0000, -0.0000,  ...,  0.0711,  0.0000,  0.0000],
        [-0.0859, -0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.6024]])

__we can calculate predictions now__

In [33]:
preds = (t_indeps * coef).sum(axis=1)
preds.shape

torch.Size([891])

__Mean absolute error(MAE) as my loss function__

In [34]:
loss = torch.abs(preds - t_dep).mean()
loss

tensor(1.3030)

**PREDICTION FUNCTION**

In [37]:
def calc_preds(indep, coef):
    return (indep * coef).sum(axis=1)

    

__LOSS FUNCTION__

In [38]:
def calc_loss(indep, dep,  coef ):
    return torch.abs(calc_preds(indep, coef) - dep).mean()
    

__diferentiating the loss function to get partial derivatives of each coefficient of the weights__

In [40]:
#but I can't just do that , I have to add grdient tracking to the coef
coef.requires_grad_()

tensor([-0.2148, -1.7572, -0.8034, -0.7240,  0.4958,  0.0252, -1.6377, -0.6625,
        -1.1374,  0.0711,  1.8761,  0.6024], requires_grad=True)

In [42]:
loss = calc_loss(t_indeps, t_dep, coef)
loss

tensor(1.3030, grad_fn=<MeanBackward0>)

In [43]:
loss.backward()

In [44]:
coef.grad

tensor([-0.2768, -0.0617, -0.0621, -0.3933, -0.3210, -0.4815, -0.1886, -0.0662,
        -0.5477, -0.2424, -0.0224, -0.5376])

__adjusting the weights with pytorch__

In [46]:
with torch.no_grad():
    coef.sub_(coef.grad * 0.01)

In [47]:
loss = calc_loss(t_indeps, t_dep, coef)
loss

tensor(1.2904, grad_fn=<MeanBackward0>)

__GREAT!!, all is perfect__

Now, I know seen how the gradient tracking works and to reduce loss>  , so lets build the model

__LINEAR REGRESSION MODEL__

In [58]:
#split into train and val split
# you can decide to use sckit learn or whatever, im going to use fastai

from fastai.data.transforms import RandomSplitter
train, val = RandomSplitter(seed=234)(df)
train, val

((#713) [594,114,274,270,514,121,857,3,130,595,209,178,439,713,371,810,191,674,339,692...],
 (#178) [586,622,246,500,450,196,0,314,28,872,323,554,407,26,889,14,536,609,102,844...])

In [60]:
X_train, y_train =  t_indeps[train], t_dep[train]
X_val , y_val =  t_indeps[val] , t_dep[val] 

len(X_train) , len(y_val)

(713, 178)

__lets try an build just one epoch__

In [61]:
#i can build an epoch which i will eventualy use in training my model
#i can just use a funtion to build update coef

def update_coef(coef, lr):
    coef.sub_(coef.grad * lr)
    

__initialize coefficients__

In [63]:
def init_params():
    return (torch.randn(n_coef)-0.5).requires_grad_()

In [88]:
def one_epoch(coef, lr):
    loss = calc_loss(X_train, y_train, coef)
    loss.backward()
    with torch.no_grad():
        update_coef(coef, lr)
    print(f'{loss : 3f}', end = " ; ")
    return coef

In [101]:
def train_model(epoch, lr):
    torch.manual_seed(12)
    coef = init_params()
    for i in range(epoch):
        one_epoch(coef, lr)
    return coef

In [102]:
coef = train_model(12, 0.02)

 2.530753 ;  2.492948 ;  2.417754 ;  2.306922 ;  2.162185 ;  1.985980 ;  1.781399 ;  1.551557 ;  1.305009 ;  1.049268 ;  0.914925 ;  0.851306 ; 

__SHOW FINAL COEFFICIENTS FOR EACH WEIGHTS__

In [119]:
def show_coef():
    return dict(zip(indep, coef.requires_grad_(False)))

In [120]:
show_coef()

{'Age': tensor(-0.1938),
 'SibSp': tensor(-1.7824),
 'Parch': tensor(-0.4656),
 'LogFare': tensor(0.6394),
 'Sex_female': tensor(0.7340),
 'Sex_male': tensor(-0.5938),
 'Embarked_C': tensor(-0.7616),
 'Embarked_Q': tensor(-1.3013),
 'Embarked_S': tensor(0.5600),
 'Pclass_1': tensor(-0.2179),
 'Pclass_2': tensor(-0.3446),
 'Pclass_3': tensor(1.0329)}

__MEASURE ACCURACY__

In [109]:
results = y_val.bool() == (calc_preds(X_val, coef) > 0.5)
results[:10]

tensor([ True, False, False, False,  True,  True, False,  True,  True,  True])

In [110]:
results.float().mean()

tensor(0.5449)

In [111]:
def accuracy():
    return (y_val.bool() == (calc_preds(X_val, coef) > 0.5)).float().mean()
    

In [113]:
accuracy()

tensor(0.5449)

__USING SIGMOID__

In [124]:
#our current preds is going below 0 ,we want to find a way to clip it between 0 and 1 

preds = calc_preds(X_val, coef)
preds[:10]

tensor([-0.2082, -0.3827,  2.4889,  1.1903, -0.4995, -0.6981,  0.9392, -0.4443,
         0.6312, -0.1480])

In [125]:
#lets solve it now

def calc_preds(indep, coef):
    preds = torch.sigmoid((indep * coef).sum(axis=1))
    return preds

In [134]:
coef = train_model(25, 0.2)

 0.376685 ;  0.376210 ;  0.375269 ;  0.373879 ;  0.372062 ;  0.369842 ;  0.367245 ;  0.364287 ;  0.360980 ;  0.357320 ;  0.353290 ;  0.348856 ;  0.343967 ;  0.338559 ;  0.332556 ;  0.325886 ;  0.318489 ;  0.310349 ;  0.301513 ;  0.292121 ;  0.282418 ;  0.272737 ;  0.263455 ;  0.254920 ;  0.247393 ; 

In [136]:
accuracy()

tensor(0.7921)